## 하나의 TFRecode 만들기

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm.notebook import tqdm
from PIL import Image
import io


############ Tensorflow에서 제공된 Type별 Feature 생성 코드 ############

def _bytes_feature(value):
    # string / byte 타입을 받아서 byte list를 리턴.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    # float / double 타입을 받아서 float list를 리턴
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _floatarray_feature(array):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=array))

def _int64_feature(value):
    # bool / enum / int / uint 타입을 받아서 int64 list를 리턴
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


############ path ############        

df_train = pd.read_csv('./data/train_df.csv')
df_test = pd.read_csv('./data/test_df.csv')

train_src = './data/train_full/train/'
valid_src = './data/train_full/validation/'
test_src = './data/test/'

path_dir = './data/train_ex'
label_list = sorted(os.listdir(path_dir)) # label 리스트
label_dict = { l_name: n for n, l_name in enumerate(label_list)}

############ DataFrame 생성 ############   

def labeling(img_name):
    img_label = df_train[df_train['file_name']==img_name].iloc[0]['label']
    return label_dict[img_label]

train_data_df = pd.DataFrame(os.listdir(train_src),
                             columns=['filename'])
valid_data_df = pd.DataFrame(os.listdir(valid_src),
                             columns=['filename'])

train_data_df['label'] = train_data_df['filename'].map(lambda x : labeling(x))
valid_data_df['label'] = valid_data_df['filename'].map(lambda x : labeling(x))


train_data_df = train_data_df.astype({'label': 'int'})
valid_data_df = valid_data_df.astype({'label': 'int'})


train_data_df['filename']  = train_data_df['filename'].map(lambda x : train_src + x)
valid_data_df['filename']  = valid_data_df['filename'].map(lambda x : valid_src + x)


display(train_data_df)
display(valid_data_df)

,filename,label
0,./data/train_full/train/10478.png,83
1,./data/train_full/train/10887.png,86
2,./data/train_full/train/10912.png,0
3,./data/train_full/train/10914.png,53
4,./data/train_full/train/11060.png,80
...,...,...
2783,./data/train_full/train/14272.png,72
2784,./data/train_full/train/14273.png,72
2785,./data/train_full/train/14274.png,28
2786,./data/train_full/train/14275.png,84


,filename,label
0,./data/train_full/validation/10000.png,72
1,./data/train_full/validation/10001.png,15
2,./data/train_full/validation/10002.png,72
3,./data/train_full/validation/10003.png,76
4,./data/train_full/validation/10004.png,3
...,...,...
1484,./data/train_full/validation/12827.png,29
1485,./data/train_full/validation/12904.png,70
1486,./data/train_full/validation/13029.png,82
1487,./data/train_full/validation/13233.png,38


In [2]:
############ TFRecord 생성 함수 정의 ############        
        
def to_tfrecords(id_list, label_list, tfrecords_name):
    # id_list : 이미지파일명을 가지고 있는 list
    # label_list : 이미지파일의 label을 가지고 있는 list
    # tfrecords_name : tfrecord의 이름(train,validation 구별을 위해필요)
    
    print("Converting 시작 (" + os.path.join(tfrecords_name + '.tfrecords') + ' 파일생성)')
    # GZIP으로 압축한 TFRecord 생성하기 위한 option
    options = tf.io.TFRecordOptions(compression_type = 'GZIP')
        
    with tf.io.TFRecordWriter(path=os.path.join(tfrecords_name + '.tfrecords'), 
                              options=options) as writer:
        
        # tqdm은 반복문의 진행상황을 progressbar를 보여주는 모듈
        # tqdm 파라미터

        # iterable : 반복자 객체
        # desc : 진행바 앞에 텍스트 출력
        # total : int, 전체 반복량
        # leave : bool, default로 True. (진행상태 잔상이 남음)
        # ncols : 진행바 컬럼길이. width 값으로 pixel 단위로 보임.
        # mininterval, maxinterval : 업데이트 주기. 
        #                            기본은 mininterval=0.1 sec, maxinterval=10 sec
        # miniters : Minimum progress display update interval, in iterations.
        # ascii : True로 하면 '#'문자로 진행바가 표시됨.
        # initial : 진행 시작값. 기본은 0
        # bar_format : str

        # tqdm method
        # clear() : 삭제
        # refresh() : 강제 갱신
        
        for id_, label_ in tqdm(zip(id_list, label_list), 
                                total=len(id_list), 
                                position=0, 
                                leave=True):
            image_path = id_
            
            # TFRecord를 생성하는 시점에 resize를 할수도 있고 TFRecord를 읽어서 DataSet을 만들때
            # resize를 할 수 도 있다. 
            # resize 된 이미지를 저장할 경우 처리에 주의해야 한다. 
            # 잘못하는 경우 이미지 데이터가 깨져서 저장될 수 있음.
            # 일반적으로 원본을 저장하고 사용할 때 resize해서 사용하는것이 일반적임.
            
            # 원본 이미지를 resize하지 않고 TFRecord로 저장.
            _binary_image = tf.io.read_file(image_path)
            
            # Example 객체 생성
            # 파이썬의 문자열은 모두 unicode. unicode 문자열을 byte배열로 바꾸는 함수가 encode()
            # 각 type에 맞게 Feature 객체 생성 후 dict 생성. 
            # 이 dict를 이용해 Feauture 객체 생성 후 Example 객체 생성.
            string_set = tf.train.Example(features=tf.train.Features(feature={
                'image_raw': _bytes_feature(_binary_image),
                'label': _int64_feature(label_)                
            }))

            # 만들어진 Example 객체를 binary string으로 변환한 후 파일에 저장
            writer.write(string_set.SerializeToString())    
            

In [6]:
############ 데이터 분리 및 TFRecord 생성 함수 호출 ############        

from sklearn import utils

# train_ids, val_ids, train_labels, val_labels = \
#     train_test_split(df['filename'], 
#                      df['label'], 
#                      test_size=0.2, 
#                      random_state=40, 
#                      shuffle=True)


# to_tfrecords(train_ids, train_labels, './cat_dog_train')
# to_tfrecords(val_ids, val_labels, './cat_dog_valid')

train_shuffle_df = utils.shuffle(train_data_df)
valid_shuffle_df = utils.shuffle(valid_data_df)

data_dir = './data/class_tf'
os.mkdir(data_dir)

to_tfrecords(train_shuffle_df['filename'], train_shuffle_df['label'], './data/class_tf/train_0502')
to_tfrecords(valid_shuffle_df['filename'], valid_shuffle_df['label'], './data/class_tf/valid_0502')

Converting 시작 (./data/class_tf/train_0502.tfrecords 파일생성)


  0%|          | 0/2788 [00:00<?, ?it/s]

Converting 시작 (./data/class_tf/valid_0502.tfrecords 파일생성)


  0%|          | 0/1489 [00:00<?, ?it/s]